##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Title

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/abhichou4/addons/blob/tutorialAvg/docs/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/abhichou4/addons/blob/tutorialAvg/docs/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This notebook demonstrates how to use Moving Average Optimizer along with the Model Average Checkpoint from tensorflow addons pagkage.




## Moving Averaging 

> The advantage of Moving Averaging is that they are less prone to rampant loss shifts or irregular data representation in the latest batch. It gives a smooothened and a more genral idea of the model training until some point.

## Stocastic Averaging

> Stocastic Weight Averaging converges to wider optimas. By doing so, it resembles geometric ensembeling. SWA is a simple method to improve model performance when used as a wrapper around other optimizers and averaging results from different points of trajectory of the inner optimizer.

## Model Average Checkpoint 

> ```callbacks.ModelCheckpoint``` doesn't give you the option to save moving average weights in the middle of traning, which is why Model Average Optimizers required a custom callback. Using the ```update_weights``` parameter, ```ModelAverageCheckpoint``` allows you to:
1.   Assign the moving average weights to the model, and save them.
2.   Keep the old non-averaged weights, but the saved model uses the average weights.

## Setup

In [2]:
try:
    %tensorflow_version 2.x
except:
    pass

import tensorflow as tf
import tensorflow_addons as tfa

TensorFlow 2.x selected.


In [0]:
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt
import os

## Build Model 

In [0]:
def create_model(opt):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),                         
        tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
        tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model

## Prepare Dataset

In [0]:
#Load Fashion MNIST dataset
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

We will be comparing three optimizers here:

*   Unwrapped SGD
*   SGD with Moving Average
*   SGD with Stochastic Weight Averaging

And see how they perform with the same model.

In [0]:
#Optimizers 
sgd = tf.keras.optimizers.SGD(0.01)
moving_avg_sgd = tfa.optimizers.MovingAverage(sgd)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

Both ```MovingAverage``` and ```StocasticAverage``` optimers use ```ModelAverageCheckpoint```.

In [0]:
#Callback 
checkpoint_path = "./training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath=checkpoint_dir, 
                                                    update_weights=True)

## Train Model


### Vanilla SGD Optimizer 

In [8]:
#Build Model
model = create_model(sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
1868/1875 [============================>.] - ETA: 0s - loss: 0.7708 - accuracy: 0.7461
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.7700 - accuracy: 0.7463
Epoch 2/5
1850/1875 [============================>.] - ETA: 0s - loss: 0.5062 - accuracy: 0.8236
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5058 - accuracy: 0.8238
Epoch 3/5
1852/1875 [============================>.] - ETA: 0s - loss: 0.4590 - accuracy: 0.8386
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4590 - accuracy: 0.8385
Epoch 4/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.4305 - accuracy: 0.8501
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4306 - accuracy: 0.8500
Epoch 5/5
1866/1875 [============================>.] - ETA: 0s - lo

In [9]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 79.5507 - accuracy: 0.8011
Loss : 79.5506900449872
Accuracy : 0.8011


### Moving Average SGD

In [10]:
#Build Model
model = create_model(moving_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1870/1875 [============================>.] - ETA: 0s - loss: 0.7541 - accuracy: 0.7524
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 5s 2ms/step - loss: 0.7535 - accuracy: 0.7525
Epoch 2/5
1867/1875 [============================>.] - ETA: 0s - loss: 0.4944 - accuracy: 0.8290
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4944 - accuracy: 0.8291
Epoch 3/5
1873/1875 [============================>.] - ETA: 0s - loss: 0.4485 - accuracy: 0.8429
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4483 - accuracy: 0.8430
Epoch 4/5
1869/1875 [============================>.] - ETA: 0s - loss: 0.4233 - accuracy: 0.8519
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4233 - acc

In [11]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 86.9890 - accuracy: 0.7955
Loss : 86.98903251190185
Accuracy : 0.7955


### Stocastic Weight Average SGD 

In [12]:
#Build Model
model.load_weights(checkpoint_dir)
model = create_model(stocastic_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
1867/1875 [============================>.] - ETA: 0s - loss: 0.7787 - accuracy: 0.7398
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 6s 3ms/step - loss: 0.7776 - accuracy: 0.7401
Epoch 2/5
1867/1875 [============================>.] - ETA: 0s - loss: 0.5027 - accuracy: 0.8231
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5027 - accuracy: 0.8232
Epoch 3/5
1868/1875 [============================>.] - ETA: 0s - loss: 0.4602 - accuracy: 0.8376
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4598 - accuracy: 0.8378
Epoch 4/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.4335 - accuracy: 0.8478
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4334 - accuracy: 0.8478
Epoch 5/5
1858/1875 [============================>.] - ETA: 0s - lo

In [13]:
#Evalute results
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 85.3121 - accuracy: 0.7944
Loss : 85.31213686828613
Accuracy : 0.7944
